In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# some basic imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns

In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Jordan/Data"

In [ ]:
#changing the directory to the data directory
%cd /content/drive/My\ Drive/Jordan/Data

In [ ]:
class DataShell:
    
    def __init__(self):
      """Initializes the class with some information about the data and the model"""
      self.data_used = 'The data used is the transformed insurance data'
      

    def read_data(self,filename):
        
        """Reads the data from the current directory
           parameters: filename in the directory
           returns: Dataframe of the data
        """
        self.data = pd.read_stata(filename)
        return self.data

    def get_initial_data(self):
        """Returns the initial dataframe in its current form"""
        return self.data
    
    def var_init(self):
        self.data['Switch_down'] = 0
        self.data['Switch_up'] = 0
        self.data['Addition'] = 0
        self.data['VIP_Plan'] = 0
        self.data['Economy_Plan'] = 0
        self.data['Stay_high'] = 0
        self.data['Stay_low'] = 0
    
    def replace_data(self):
        self.data1 = self.data
        
    def clean_data(self):
        self.data1['endo_type'].replace('', np.nan, inplace=True)
        self.data1['network_type'].replace('', np.nan, inplace=True)
        self.data1.dropna(subset=['endo_type','network_type'], inplace=True)
        
    def network_coding(self):
        vip = ['General Network','Comprehensive Network']
        eco = ['Super-Restricted Network','Restricted Network','Workers Network']
        for i in range(len(self.data1)):
            if self.data1.loc[i,'network_type'] in vip:
                self.data1.loc[i,'VIP_Plan'] = 1       
            else:
                self.data1.loc[i,'Economy_Plan'] = 1
                
        return self.data1
    
    def switch_coding(self):
        self.data2 = pd.DataFrame(columns=self.data1.columns)
        for i,name in enumerate(self.data1['principal_hof'].unique()):
            print(i/len(self.data1['principal_hof'].unique())*100)
            k = self.data1[self.data1['principal_hof']==name]
            if len(k['endo_type'].unique()) == 1:
                if 'Initial New' in k['endo_type'].unique():
                    self.data2 = self.data2.append(k)
                    continue
            if ('Voluntary Deletion' or 'Replacement of Lost Card') in k['endo_type'].unique():
                k['Switch_down'] = 1
            if ('Change In Category' or 'Error Correction' or 'Elective Cancellation') in k['endo_type'].unique():
                k['Switch_up'] = 1
            if 'Addition' in k['endo_type'].unique():
                k['Addition'] = 1
            self.data2 = self.data2.append(k)
        return self.data2

In [ ]:
#Creates an object for the class and reads the data from the directory
data_shell = DataShell()
data = data_shell.read_data('merged_all1415.dta')
#data.head()
print("The data has ",data.shape[0]," rows and ",data.shape[1]," atrributes")

The data has  633516  rows and  80  atrributes


In [ ]:
data.head()

,payer,master_contract,contract,policynumber,effictive_date,exp_date,endorsment_id,endo_type,endo_date,issue_date,...,po,orderdate,cheque_nbr,settledate,status,claimcatogory,icdchapter,icd_subchapter,assessment,recoverynote
0,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,...,108686.0,NaN,,,PO Issued,To Be Paid,Diseases of the musculoskeletal system and con...,Other joint disorders (M20-M25) (734-736.9) (7...,"719.47 Pain in joint, ankle and foot",NaN
1,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,...,106714.0,NaN,197865,2/4/2016,Settled,Paid,Diseases of the genitourinary system (ICD9CM 5...,Renal tubulo-interstitial diseases (N10-N16) (...,593.89 Other specified disorder of kidney and ...,NaN
2,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,...,107092.0,NaN,200400,2/22/2016,Settled,Paid,Diseases of the musculoskeletal system and con...,Other joint disorders (M20-M25) (734-736.9) (7...,"719.47 Pain in joint, ankle and foot",NaN
3,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,...,105824.0,NaN,175021,11/5/2015,Settled,Paid,"Symptoms, signs and abnormal clinical and labo...",Symptoms and signs involving the digestive sys...,"789.00 Abdominal pain, unspecified site",NaN
4,Al Hilal Takaful,Global Force Capital Group,Global Force Capital Group,100-2-NS-X-1071,8/24/2015,8/24/2016,1.0,Initial New,24-Aug-15,31-Aug-15,...,105899.0,NaN,194075,1/21/2016,Settled,Paid,"Injury, poisoning and certain other consequenc...",Fracture Of Lower Limb (820-829),825.23 Closed fracture of cuboid bone,NaN


In [ ]:
data_shell.var_init()
data_shell.replace_data()
data_shell.clean_data()

In [ ]:
data1.groupby(["endo_type", "Size"]).size().reset_index(name="Time")

In [ ]:
data1 = data_shell.network_coding()

KeyboardInterrupt: 

In [ ]:
if ('Voluntary Deletion' or 'Replacement of Lost Card') in k['endo_type'].unique():
  k['switch_down'] = 1
if ('Change In Category' or 'Error Correction' or 'Elective Cancellation') in k['endo_type'].unique():
  k['switch_up'] = 1

In [ ]:
data['Switch_down'] = 0
data['Switch_up'] = 0
data['Addition'] = 0
data1 = pd.DataFrame(columns=data.columns)
for j,i in enumerate(data['principal_hof'].unique()):
  if j == 3:
    break
  k = data[data['principal_hof']==i]
  if len(k['endo_type'].unique()) == 1:
    if 'Initial New' in k['endo_type'].unique():
      data1 = data1.append(k)
      continue
  if ('Voluntary Deletion' or 'Replacement of Lost Card') in k['endo_type'].unique():
    k['Switch_down'] = 1
  if ('Change In Category' or 'Error Correction' or 'Elective Cancellation') in k['endo_type'].unique():
    k['Switch_up'] = 1
  if 'Addition' in k['endo_type'].unique():
    k['Addition'] = 1
  data1 = data1.append(k)

In [ ]:
data['VIP_Plan'] = 0
data['Economy_Plan'] = 0
vip = ['General Network','Comprehensive Network']
eco = ['Super-Restricted Network','Restricted Network','Workers Network']
for i in range(len(data)):
    if data.loc[i,'network_type'] in vip:
        data[i,'VIP_Plan'] = 1
    elif data.loc[i,'network_type'] in eco:
        data.loc[i,'Economy_Plan'] = 1
    else:
        pass

In [ ]:
data.loc[1849,'network_type'] in ['General Network','Comprehensive Network']